In [ ]:
import os
from infoclus import InfoClus


RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')
DATA_SET_NAME = 'german_socio_eco'

infoclus = InfoClus(DATA_SET_NAME, RELATIVE_DATA_PATH)

: 

In [12]:
import os
script_a_dir = os.path.dirname(os.path.abspath(__))
path = os.path.join(script_a_dir, 'data')

TypeError: abspath() missing 1 required positional argument: 'path'